In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *


fig_DIR = "../figs/team_ratings/"
box_DIR = "../data/boxscores_team/"

In [ ]:
dfb = []
for season in range(2004,2024):
    df = pd.read_csv(box_DIR + f"NBA_BoxScores_Standard_{season}.csv")
    df['Team_Split'] = df['MATCHUP'].str.replace("@","")
    df['Team_Split'] = df['Team_Split'].str.replace("vs.","")
    df[['Team1','Team2']] = df['Team_Split'].str.split("  ",expand=True)
    df1 = df.groupby("TEAM_NAME")[["TEAM_NAME","GAME_DATE","PLUS_MINUS"]]
    keys = list(df1.groups)
    dfa = []
    for key in keys:
        df1t = df1.get_group(key)
        df1t["PM"] = df1t["PLUS_MINUS"].cumsum()
        df1t = df1t.reset_index(drop=True).reset_index()
        df1t["index"] +=1
        df1t = df1t.rename(columns={"index":"Games","TEAM_NAME":"Team"}).drop(columns = ["PLUS_MINUS"])
        dfa.append(df1t)
    df2 = pd.concat(dfa)
    df2["Season"] = season + 1
    dfb.append(df2)
df3 = pd.concat(dfb)
df3 = df3[["Season","Team","Games","PM"]]

In [ ]:
n = 29
df4 = df3.query(f"Games < {n}").reset_index(drop=True)

In [ ]:
dff = df4.query(f"Games == {n-1}").sort_values("PM",ascending=False).head(10)

In [ ]:
df5 = pd.merge(df4,dff,how="inner",on=["Season","Team"],suffixes=["","_x"]).drop(columns=["Games_x","PM_x"])

In [ ]:
df5["ST"] = df5["Season"].astype(str) + " " + df5["Team"]

In [ ]:
import matplotlib as mpl

In [ ]:
df_teams = pd.read_csv("../data/NBA_teams_colors_logos.csv")

In [ ]:
df_teams["Team"] =df_teams["nameTeam"]

In [ ]:
df6 = pd.merge(df5,df_teams,on="Team")

In [ ]:
cmap=mpl.colormaps["tab10"].resampled(10)

In [ ]:
colors =[]
for i in np.linspace(0,1,10):
    print(cmap(i))
    colors.append(cmap(i))

In [ ]:
p = (
    ggplot(df6,aes(x="Games",y="PM",color="ST"))
    + geom_line(size=1)
    # + scale_color_gradientn(colors=colors)
    # + theme(legend_title=element_blank())
    + theme_idv
    + theme(figure_size=(8,5))
    + labs(
        title = "Teams with Best +/- Through 28 Games",
        subtitle = "Over the Past 20 Seasons",
        caption = "@sradjoker | Source = nba.com/stats",
    )
    + theme(
        legend_title=element_blank()
    )
)
p.draw()
p.save("Best_pm.png",dpi=300)